<a href="https://colab.research.google.com/github/IsaacAIML2023/-Detecting-steganography-with-tools-like-StegExpose-analyzing-file-signatures/blob/main/hybrid_surgical_readiness_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surgical Readiness Prediction Using Hybrid ML Models
This notebook loads a synthetic trauma dataset, performs preprocessing, and trains logistic regression, LightGBM, and LSTM models.

In [3]:
!pip install lightgbm --quiet
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [4]:
df = pd.read_csv('synthetic_trauma_dataset.csv', parse_dates=['Timestamp'])
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'synthetic_trauma_dataset.csv'

In [ ]:
df.sort_values(by=['PatientID', 'Timestamp'], inplace=True)
features = ['HR', 'RR', 'SpO2', 'Temp', 'Lactate', 'pH', 'BaseExcess', 'TimeSinceInjury']
X = df[features]
y = df['ReadyFlag']
X.fillna(method='ffill', inplace=True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
lr_model = LogisticRegression(max_iter=500)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_lr))


In [ ]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_lgbm))


In [ ]:
# Normalize features for LSTM
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

seq_length = 10
generator = TimeseriesGenerator(X_scaled, y.values, length=seq_length, batch_size=64)

# LSTM Model
lstm_model = Sequential([
    LSTM(64, activation='relu', input_shape=(seq_length, X.shape[1])),
    Dense(1, activation='sigmoid')
])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(generator, epochs=5)
